In [1]:
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
from random import shuffle

# Import time series data

In [2]:
import pandas as pd
df = pd.read_csv("yahoo_stock.csv")
df

,Date,High,Low,Open,Close,Volume,Adj Close
0,2015-11-23,2095.610107,2081.389893,2089.409912,2086.590088,3.587980e+09,2086.590088
1,2015-11-24,2094.120117,2070.290039,2084.419922,2089.139893,3.884930e+09,2089.139893
2,2015-11-25,2093.000000,2086.300049,2089.300049,2088.870117,2.852940e+09,2088.870117
3,2015-11-26,2093.000000,2086.300049,2089.300049,2088.870117,2.852940e+09,2088.870117
4,2015-11-27,2093.290039,2084.129883,2088.820068,2090.110107,1.466840e+09,2090.110107
...,...,...,...,...,...,...,...
1820,2020-11-16,3628.510010,3600.159912,3600.159912,3626.909912,5.281980e+09,3626.909912
1821,2020-11-17,3623.110107,3588.679932,3610.310059,3609.530029,4.799570e+09,3609.530029
1822,2020-11-18,3619.090088,3567.330078,3612.090088,3567.790039,5.274450e+09,3567.790039
1823,2020-11-19,3585.219971,3543.840088,3559.409912,3581.870117,4.347200e+09,3581.870117


#Create numpy array 

In [4]:
value = df['Adj Close']
values = np.array(value)
sample = shuffle(value)

C:\ProgramData\Anaconda3\lib\random.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[i], x[j] = x[j], x[i]


In [6]:
sample = np.array(value)
sample = sample[0:300]

In [7]:
sample,values

(array([2926.45996094, 3465.38989258, 2682.62011719, 2881.39990234,
        2792.66992188, 2132.97998047, 1864.7800293 , 2052.32006836,
        3351.60009766, 2633.08007812, 3289.29003906, 2793.89990234,
        2695.94995117, 2873.34008789, 2943.0300293 , 2788.86010742,
        2716.26000977, 3545.5300293 , 3240.02001953, 3080.82006836,
        1880.05004883, 2718.37011719, 2047.63000488, 2409.38989258,
        2922.94995117, 2789.82006836, 1853.43994141, 2357.48999023,
        2743.79003906, 2384.19995117, 2351.15991211, 2168.47998047,
        2328.94995117, 2464.61010742, 3097.73999023, 2666.93994141,
        3271.12011719, 2803.68994141, 2170.06005859, 3295.4699707 ,
        2952.01000977, 2141.15991211, 3179.7199707 , 2263.68994141,
        2263.79003906, 2752.01000977, 2950.45996094, 2816.29003906,
        2549.33007812, 2256.95996094, 2907.06005859, 3253.05004883,
        2390.89990234, 2164.68994141, 2670.29003906, 3153.62988281,
        2561.26000977, 3117.86010742, 2717.07006

In [8]:
# Create random samples

norm_A = values
norm_B = sample

In [9]:
norm_A

array([2086.59008789, 2089.13989258, 2088.87011719, ..., 3567.79003906,
       3581.87011719, 3557.54003906])

# Create Python Script for K-S Test

In [10]:
def ks_2samp(sample1, sample2):
    # Gets all observations
    observations = np.concatenate((sample1, sample2))
    observations.sort()
    # Sorts the samples
    sample1.sort()
    sample2.sort()
    # Evaluates the KS statistic
    D_ks = [] # KS Statistic list
    for x in observations:
        cdf_sample1 = cdf(sample = sample1, x  = x)
        cdf_sample2 = cdf(sample = sample2, x  = x)
        D_ks.append(abs(cdf_sample1 - cdf_sample2))
    ks_stat = max(D_ks)
    # Calculates the P-Value based on the two-sided test
    # The P-Value comes from the KS Distribution Survival Function (SF = 1-CDF)
    m, n = float(len(sample1)), float(len(sample2))
    en = m * n / (m + n)
    p_value = stats.kstwo.sf(ks_stat, np.round(en))
    return {"ks_stat": ks_stat, "p_value" : p_value}


In [11]:
def cdf(sample, x, sort = False):
    # Sorts the sample, if unsorted
    if sort:
        sample.sort()
    # Counts how many observations are below x
    cdf = sum(sample <= x)
    # Divides by the total number of observations
    cdf = cdf / len(sample)
    return cdf

In [12]:
sets = [norm_A, norm_B, ]
names = ['norm_A', 'norm_B']
ks_scores = {}
for _ in range(len(names)):
    name1 = names.pop(0)
    sample1 = sets.pop(0)
    for name2, sample2 in zip(names, sets):
        key1 = name1 + "_" +  name2
        key2 = name2 + "_" +  name1
        ks = ks_2samp(sample1, sample2)
        ks_scores[key1] = ks
        ks_scores[key2] = ks
# Prints the results
print(f"norm_A vs norm_B: ks = {ks_scores['norm_A_norm_B']['ks_stat']:.4f} (p-value = {ks_scores['norm_A_norm_B']['p_value']:.3e}, are equal = {ks_scores['norm_A_norm_B']['p_value'] > 0.05})")


norm_A vs norm_B: ks = 0.0452 (p-value = 6.517e-01, are equal = True)
